In [51]:
import gamv
import pygslib
import pandas as pd
import numpy as np

In [52]:
data = pygslib.gslib.read_gslib_file('../pygslib/data/cluster.dat')

In [53]:
data['Zlocation']= 0.0
data['Domain'] = 0
data.head()

,Xlocation,Ylocation,Primary,Secondary,Declustering Weight,Zlocation,Domain
0,39.5,18.5,0.06,0.22,1.619,0.0,0
1,5.5,1.5,0.06,0.27,1.619,0.0,0
2,38.5,5.5,0.08,0.40,1.416,0.0,0
3,20.5,1.5,0.09,0.39,1.821,0.0,0
4,27.5,14.5,0.09,0.24,1.349,0.0,0


In [54]:
#-----------------------------------------------------------------------------------------------------------------
#
#    Variograms GAMV low level functions
#
#-----------------------------------------------------------------------------------------------------------------

def gamv(parameters):
    """Calculate experimental variograms on sparse data

    This function is similar to the GSLIB gamv function but with some minor differences:

     - the indicator automatic transformation is not applied; you may define
       indicator variables externally.
     - a bhid variable is always required, to avoid downhole variogram use
       ``bhid=None`` or ``bhid = 0``.
     - Z variable is required, you can use constant coordinate to reduce
       dimensions, for example use Z=None in the parameter file or
       Z = 0 in the file.
     - The variogram cloud was implemented but it only works for the first
       variogram and first direction. It only works for some variogram types.


    Parameters
    ----------
        parameters  :  dict
            dictionary with calculation parameters

    The dictionary with parameters may be as follows::


            parameters = {
                    'data'   :  data,          # Pandas dataframe
                    'columns':  ['X','Y','Z','BHID'],  # X,Y,Z, and BHID column names
                    'vr'     :  ['v1','v2'],            # Variables names 
                    'tmin'   : -1.0e21,        # trimming limits, float, if None or basent = -1.0e21
                    'tmax'   :  1.0e21,        # trimming limits, float, if None or basent = 1.0e21
                    'nlag'   :  10,            # number of lags, int
                    'xlag'   :  4,             # lag separation distance, float
                    'xltol'  :  2,             # lag tolerance, float
                    'azm'    : [0,0,90],       # azimuth, array('f') with bounds (ndir)
                    'atol'   : [90,22.5,22.5], # azimuth tolerance, array('f') with bounds (ndir)
                    'bandwh' : [50,10,10],     # bandwidth 'horizontal', array('f') with bounds (ndir)
                    'dip'    : [0,0,0],        # dip, array('f') with bounds (ndir)
                    'dtol'   : [10,10,10],     # dip tolerance, array('f') with bounds (ndir)
                    'bandwd' : [10,10,10],     # bandwidth 'vertical', array('f') with bounds (ndir)
                    'isill'  : 0,              # standardize sills? (0=no, 1=yes), int
                    'sills'  : [100, 200],     # variance used to std the sills, array('f') with bounds (nv)
                    'ivtail' : 1,# tail var., array('i') with bounds (nvarg), nvarg is number of variograms
                    'ivhead' : 1,# head var., array('i') with bounds (nvarg)
                    'ivtype' : 1,# variogram type, array('i') with bounds (nvarg)
                    'maxclp' : 50000}          # maximum number of variogram point cloud to use, input int

    Warnings
    --------
    bhid must be an array of integers

    Returns
    -------
       pdis :  Distance of pairs falling into this lag
       pgam :  Semivariogram, covariance, correlogram,... value
       phm  :  Mean of the tail data
       ptm  :  Mean of the head data
       phv  :  Variance of the tail data
       ptv  :  Variance of the head data
       pnump:  Number of pairs
       cldi :  data index of the head
       cldj :  data index of the tail
       cldg :  Semivariogram, covariance, ... value
       cldh :  Distance of each pair


    Note
    -----
    The output variables with prefix *cld* are for variogram cloud and
    with prefix *p* are for directional variograms

    The variables with prefix *p* are similar to the output generated
    by the GSLIB standalone program **gamv**.

    The variogram cloud only works for the first variogram/direction
    and only if the variogram of type 1, 2, 6, 7 and 8.

    The variogram types are:

     - traditional semivariogram (1)
     - traditional cross semivariogram (2)
     - pairwise relative semivariogram (6)
     - semivariogram of logarithms (7)
     - semimadogram(8)

    """

    # we only use variograms from 1 to 8, gslib uses to 10
    assert (parameters['ivtype']>=0 and parameters['ivtype']<=8)
    
    newpar = {}
    newpar['x'] = parameters['data'][parameters['columns'][0]].values
    newpar['y'] = parameters['data'][parameters['columns'][1]].values
    newpar['z'] = parameters['data'][parameters['columns'][2]].values
    newpar['bhid'] = parameters['data'][parameters['columns'][3]].values
    newpar['vr'] = parameters['data'][parameters['vr']].values
    if 'tmin' in parameters:
        if parameters['tmin'] is not None:
            newpar['tmin'] = parameters['tmin']
        else:
            newpar['tmin'] = -1.0e21
    else:
        newpar['tmin'] = -1.0e21
        
    if 'tmax' in parameters:
        if parameters['tmax'] is not None:
            newpar['tmax'] = parameters['tmax']
        else:
            newpar['tmax'] = 1.0e21
    else:
        newpar['tmax'] = 1.0e21  
    
    if 'sills' in parameters:
        if parameters['sills'] is not None:
            newpar['sills'] = parameters['sills']
        else:
            newpar['sills'] = []
            for ivr in parameters['vr']:
                newpar['sills'].append(parameters['data'][ivr].var())
    else:
        newpar['sills'] = []
        for ivr in parameters['vr']:
            newpar['sills'].append(parameters['data'][ivr].var())     
    
    newpar['nlag'] = parameters['nlag'] 
    newpar['xlag'] = parameters['xlag'] 
    newpar['xltol'] = parameters['xltol'] 
    newpar['azm'] = parameters['azm'] 
    newpar['atol'] = parameters['atol'] 
    newpar['bandwh'] = parameters['bandwh'] 
    newpar['dip'] = parameters['dip'] 
    newpar['dtol'] = parameters['dtol'] 
    newpar['bandwd'] = parameters['bandwd']
    newpar['isill'] = [parameters['isill']] 
    newpar['ivtail'] = [parameters['ivtail']]    
    newpar['ivhead'] = [parameters['ivhead']] 
    newpar['ivtype'] = [parameters['ivtype']]
    newpar['maxclp'] = parameters['maxclp']
    
    npt,dis, gam, hm, tm, hv, tv, cldi, cldj, cldg, cldh, l = pygslib.gslib.__variograms.gamv(**newpar)

    if l==parameters['maxclp']:
        warnings.warn( 'Warning: l == maxclp; maximum number ' + \
                       'of point clouds reached, increase maxclp' + \
                       'or modify variogram parameters')

    #remove crap data from variogram cloud
    cldi=cldi[:l]
    cldj=cldj[:l]
    cldg=cldg[:l]
    cldh=cldh[:l]

    # get output like in gslib

    ndir = len(parameters['azm'])
    nlag = parameters['nlag']


    pdis,pgam, phm,ptm,phv,ptv,pnump = pygslib.gslib.__variograms.writeout(1,ndir,nlag,npt,dis,gam,hm,tm,hv,tv)
    
    # each is an array of (nvarg, ndir, nlag+2)
    lag = []
    azm = []
    dip = []
    vtail= []
    vhead= []
    vtype= []
    dirID = []

    for idr in range(ndir):

        lag.append(0)
        lag.append(0.5)

        azm.append(parameters['azm'][idr])
        dip.append(parameters['dip'][idr])
        azm.append(parameters['azm'][idr])
        dip.append(parameters['dip'][idr])

        vtail.append(parameters['vr'][parameters['ivtail']-1])
        vhead.append(parameters['vr'][parameters['ivhead']-1])
        vtype.append(parameters['ivtype'])
        vtail.append(parameters['vr'][parameters['ivtail']-1])
        vhead.append(parameters['vr'][parameters['ivhead']-1])
        vtype.append(parameters['ivtype'])   

        dirID.append(idr)
        dirID.append(idr)          


        for ilg in range(2,nlag+2):
            lag.append(ilg-1)
            azm.append(parameters['azm'][idr])
            dip.append(parameters['dip'][idr])
            vtail.append(parameters['vr'][parameters['ivtail']-1])
            vhead.append(parameters['vr'][parameters['ivhead']-1])
            vtype.append(parameters['ivtype'])
            dirID.append(idr)
                
                
    vtab = pd.DataFrame(
        {
            'Dip dir': azm,
            'Dip': dip,
            'Head' : vhead,
            'Tail' : vtail,
            'Lag distance': pdis.ravel()*0.0,
            'X': pdis.ravel()*0.0,
            'Y': pdis.ravel()*0.0,
            'Z': pdis.ravel()*0.0,
            'Average distance': pdis.ravel(), 
            'Spatial function value': pgam.ravel(), 
            'Spatial function type' : vtype,
            'Mean head': phm.ravel(),
            'Mean tail': ptm.ravel(),
            'Variance head': phv.ravel(),
            'Variance tail': ptv.ravel(),
            'Number of pairs': pnump.ravel(),
            'Dir ID': dirID,
            'Lag': lag,
        }
    )

    DEG2RAD=3.14159265/180.0
    
    vtab['Lag distance'] = vtab['Lag']*parameters['xlag'] 
    vtab['X'] = np.sin(DEG2RAD*vtab['Dip dir'])*np.cos(DEG2RAD*vtab['Dip'])*vtab['Lag distance'] 
    vtab['Y'] = np.cos(DEG2RAD*vtab['Dip dir'])*np.cos(DEG2RAD*vtab['Dip'])*vtab['Lag distance']
    vtab['Z'] =                                 np.sin(DEG2RAD*vtab['Dip'])*vtab['Lag distance'] 
    
    vtab.loc[vtab['Lag distance']==0, 'Spatial function value']=np.nan
    vtab.loc[vtab['Number of pairs']==0, 'Spatial function value']=np.nan
    
    cloud = pd.DataFrame(
        {
           'Index head': cldi, 
           'Index tail': cldj,
           'Spatial function value': cldg,  
           'Distance': cldh  
        }
    )
    
    return vtab, cloud

In [55]:
htol = 10 
dtol = 10
wh = 100000
wd = 100000

dips = np.arange(-85,90,dtol)
azms = np.arange(-5,365,htol)
ndip = dips.shape[0]
nazm = azms.shape[0]

azm = np.ones(nazm*ndip)
dip = np.ones(nazm*ndip)

di = -1
for i in range(ndip):
    for j in range(nazm):
        di = di+1
        azm[di]=azms[j]
        dip[di]=dips[i]
        
    

atol = np.ones(nazm*ndip)*htol
dtol = np.ones(nazm*ndip)*dtol

bandwh = np.ones(nazm*ndip)*wh
bandwd = np.ones(nazm*ndip)*wd
 
parameters = {
        'data'   :  data,          # Pandas dataframe
        'columns':  ['Xlocation','Ylocation','Zlocation','Domain'],  # X,Y,Z, and BHID column names
        'vr'     :  ['Primary'],            # Variables names 
        'nlag'   :  10,            # number of lags, int
        'xlag'   :  4,             # lag separation distance, float
        'xltol'  :  2,             # lag tolerance, float
        'azm'    : azm,       # azimuth, array('f') with bounds (ndir)
        'atol'   : atol, # azimuth tolerance, array('f') with bounds (ndir)
        'bandwh' : bandwh,     # bandwidth 'horizontal', array('f') with bounds (ndir)
        'dip'    : dip,        # dip, array('f') with bounds (ndir)
        'dtol'   : dtol,     # dip tolerance, array('f') with bounds (ndir)
        'bandwd' : bandwd,     # bandwidth 'vertical', array('f') with bounds (ndir)
        'isill'  : 1,              # standardize sills? (0=no, 1=yes), int
        'ivtail' : 1,# tail var., array('i') with bounds (nvarg), nvarg is number of variograms
        'ivhead' : 1,# head var., array('i') with bounds (nvarg)
        'ivtype' : 4,# variogram type, array('i') with bounds (nvarg)
        'maxclp' : 50000}          # maximum number of variogram point cloud to use, input int

In [56]:
vtab, cloud= gamv(parameters)

In [57]:
vtab.tail()

,Dip dir,Dip,Head,Tail,Lag distance,X,Y,Z,Average distance,Spatial function value,Spatial function type,Mean head,Mean tail,Variance head,Variance tail,Number of pairs,Dir ID,Lag
7987,355.0,85.0,Primary,Primary,24.0,-0.182307,2.083778,23.908673,0.0,NaN,4,0.0,0.0,0.0,0.0,0,665,6.0
7988,355.0,85.0,Primary,Primary,28.0,-0.212691,2.431075,27.893452,0.0,NaN,4,0.0,0.0,0.0,0.0,0,665,7.0
7989,355.0,85.0,Primary,Primary,32.0,-0.243076,2.778371,31.878230,0.0,NaN,4,0.0,0.0,0.0,0.0,0,665,8.0
7990,355.0,85.0,Primary,Primary,36.0,-0.273460,3.125667,35.863009,0.0,NaN,4,0.0,0.0,0.0,0.0,0,665,9.0
7991,355.0,85.0,Primary,Primary,40.0,-0.303845,3.472964,39.847788,0.0,NaN,4,0.0,0.0,0.0,0.0,0,665,10.0


In [58]:
vtab.to_csv('vtab.csv', index=False)

In [59]:
# variance and varianze from head/tail is not equal but close
print (data['Primary'].var())
vtab.loc[vtab['Lag distance']==0,'Variance head']

45.247481829393635


0       44.924286
12      44.924286
24      44.924286
36      44.924286
48      44.924286
60      44.924286
72      44.924286
84      44.924286
96      44.924286
108     44.924286
120     44.924286
132     44.924286
144     44.924286
156     44.924286
168     44.924286
180     44.924286
192     44.924286
204     44.924286
216     44.924286
228     44.924286
240     44.924286
252     44.924286
264     44.924286
276     44.924286
288     44.924286
300     44.924286
312     44.924286
324     44.924286
336     44.924286
348     44.924286
          ...    
7632    44.924286
7644    44.924286
7656    44.924286
7668    44.924286
7680    44.924286
7692    44.924286
7704    44.924286
7716    44.924286
7728    44.924286
7740    44.924286
7752    44.924286
7764    44.924286
7776    44.924286
7788    44.924286
7800    44.924286
7812    44.924286
7824    44.924286
7836    44.924286
7848    44.924286
7860    44.924286
7872    44.924286
7884    44.924286
7896    44.924286
7908    44.924286
7920    44

In [60]:
import altair as alt
alt.renderers.enable('default')

vtab['tmp'] = '-->'
vtab['tmp2'] = '|'
vtab['test'] = vtab['Dip dir'].astype(str) + vtab['tmp'] + vtab['Dip'].astype(str) + \
               vtab['tmp2'] + vtab['Dir ID'].astype(str) + \
               vtab['tmp2'] + vtab['Spatial function type'].astype(str) 

alt.Chart(vtab).mark_line().encode(
    x='Average distance',
    y='Spatial function value',
    color='test',
    tooltip = 'Number of pairs'
) + \
alt.Chart(pd.DataFrame({'x':[0,45],'y':[0.3,.3]})).mark_line().encode(
    x='x',
    y='y'
)


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

LayerChart({
  layer: [Chart({
    data:       Dip dir   Dip     Head     Tail  Lag distance         X         Y  \
    0        -5.0 -85.0  Primary  Primary           0.0 -0.000000  0.000000   
    1        -5.0 -85.0  Primary  Primary           2.0 -0.015192  0.173648   
    2        -5.0 -85.0  Primary  Primary           4.0 -0.030384  0.347296   
    3        -5.0 -85.0  Primary  Primary           8.0 -0.060769  0.694593   
    4        -5.0 -85.0  Primary  Primary          12.0 -0.091153  1.041889   
    5        -5.0 -85.0  Primary  Primary          16.0 -0.121538  1.389185   
    6        -5.0 -85.0  Primary  Primary          20.0 -0.151922  1.736482   
    7        -5.0 -85.0  Primary  Primary          24.0 -0.182307  2.083778   
    8        -5.0 -85.0  Primary  Primary          28.0 -0.212691  2.431075   
    9        -5.0 -85.0  Primary  Primary          32.0 -0.243076  2.778371   
    10       -5.0 -85.0  Primary  Primary          36.0 -0.273460  3.125667   
    11       -5

In [61]:
# get order as in a vtk structured grid 
vtab['ijk']=0
vtab.loc[vtab.sort_values(by = ['Dip','Lag','Dip dir']).index,'ijk']= np.arange(vtab.shape[0])
alt.Chart(vtab.reset_index()).mark_point().encode(
    x='X',
    y='Y',
    color='Spatial function value',
    tooltip='ijk:O'
)


MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000). For information on how to plot larger datasets in Altair, see the documentation

Chart({
  data:       index  Dip dir   Dip     Head     Tail  Lag distance         X  \
  0         0     -5.0 -85.0  Primary  Primary           0.0 -0.000000   
  1         1     -5.0 -85.0  Primary  Primary           2.0 -0.015192   
  2         2     -5.0 -85.0  Primary  Primary           4.0 -0.030384   
  3         3     -5.0 -85.0  Primary  Primary           8.0 -0.060769   
  4         4     -5.0 -85.0  Primary  Primary          12.0 -0.091153   
  5         5     -5.0 -85.0  Primary  Primary          16.0 -0.121538   
  6         6     -5.0 -85.0  Primary  Primary          20.0 -0.151922   
  7         7     -5.0 -85.0  Primary  Primary          24.0 -0.182307   
  8         8     -5.0 -85.0  Primary  Primary          28.0 -0.212691   
  9         9     -5.0 -85.0  Primary  Primary          32.0 -0.243076   
  10       10     -5.0 -85.0  Primary  Primary          36.0 -0.273460   
  11       11     -5.0 -85.0  Primary  Primary          40.0 -0.303845   
  12       12      5.0 -

In [62]:
import vtk
from vtk.numpy_interface import dataset_adapter as vtkdsa
points = vtk.vtkPoints()
dataval = []
stGrid = vtk.vtkStructuredGrid()
properties = ['Spatial function value', 'Number of pairs']
for i in range(len(properties)):
    dataval.append(vtk.vtkDoubleArray())
    dataval[i].SetName(properties[i])

for i in range(vtab.shape[0]):
    x,y,z = vtab.loc[vtab['ijk']==i,['X','Y','Z']].values[0]
    points.InsertNextPoint(x, y, z)
    values = vtab.loc[vtab['ijk']==i, properties].values[0]
    for i in range(len(properties)):
        dataval[i].InsertNextTuple([values[i]])

    
stGrid.SetDimensions(nazm,parameters['nlag']+2,ndip)
stGrid.SetPoints(points)
for i in range(len(properties)):
    stGrid.GetPointData().AddArray(dataval[i])    

writer = vtk.vtkXMLStructuredGridWriter()
writer.SetFileName('grid.vts')
writer.SetDataModeToBinary()
writer.SetInputData(stGrid)
writer.Write()

1

In [63]:
vtab.shape[0]

7992

In [64]:
a.sort()

NameError: name 'a' is not defined

In [ ]:
a.flags

In [ ]:
a